In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.9 MB/s eta 0:00:00


In [ ]:
!pip install SentencePiece # INSTALAR y luego restart el entorno de ejecución

!pip uninstall transformers accelerate
!pip install transformers[torch] accelerate


In [ ]:
#todo borrar

In [ ]:

import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch


# Load data
data_url = "/content/drive/My Drive/TFM/spell_check/data_new.csv"
df = pd.read_csv(data_url, sep=",", encoding="utf-8")

# Split data
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1)

In [ ]:
df.columns

Index(['text', 'corrected', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5'],
      dtype='object')

In [ ]:
#preprocesar

#df.drop(columns=" ID (username[number of line])")

tokenizer = T5Tokenizer.from_pretrained("t5-small")
def encode(examples):
    # Filtering out non-string values or missing values
    examples = [ex for ex in examples if isinstance(ex['text'], str) and isinstance(ex['corrected'], str)] # NAN values -- quitar

    src_texts = ["correct: " + ex['text'] for ex in examples]
    tgt_texts = [ex['corrected'] for ex in examples]
    src_encodings = tokenizer(src_texts, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    tgt_encodings = tokenizer(tgt_texts, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

    return {'input_ids': src_encodings['input_ids'], 'attention_mask': src_encodings['attention_mask'], 'labels': tgt_encodings['input_ids']}

train_encodings = encode(train_df.to_dict('records'))
val_encodings = encode(val_df.to_dict('records'))


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
#prepare dataset

class GrammarDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = GrammarDataset(train_encodings)
val_dataset = GrammarDataset(val_encodings)


In [ ]:
!pip show transformers
!pip show accelerate


Name: transformers
Version: 4.32.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Name: accelerate
Version: 0.22.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: 


In [ ]:
# check if path exists
import os
save_dir = "/content/drive/My Drive/TFM/transformers-output/spellchecker02"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
else:
  print("Yes, path exists!")


Yes, path exists!


In [ ]:
#model and training
#nota: a veces hay que reiniciar el entorno de ejecución después de instalar sentence piece, cuidado!

model = T5ForConditionalGeneration.from_pretrained("t5-small", dropout_rate=0.1) # t5-small es el original //"/path/to/last/checkpoint" es el último que se guardó


training_args = TrainingArguments(
    per_device_train_batch_size=8, #todo: probar
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=50,
    fp16=True, # it helps speeding up the training process
    evaluation_strategy="steps",
    eval_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=2, #todo: probar
    output_dir="/content/drive/My Drive/TFM/transformers-output/spellchecker02",
    num_train_epochs=3,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Step,Training Loss,Validation Loss
10,15.442800,13.699947
20,10.855800,5.190979
30,5.426100,0.811216
40,2.871100,0.194168
50,1.440400,0.198677
60,1.016000,0.237608
70,0.687900,0.275795
80,0.450900,0.301801
90,0.332200,0.260883
100,0.308900,0.206312


TrainOutput(global_step=2274, training_loss=0.2252133679146188, metrics={'train_runtime': 1159.4836, 'train_samples_per_second': 31.395, 'train_steps_per_second': 1.961, 'total_flos': 1230933684387840.0, 'train_loss': 0.2252133679146188, 'epoch': 3.0})

In [ ]:
# save model

trainer.save_model("/content/drive/My Drive/TFM/transformers-models/spellcheckerT5v0")

In [ ]:
!pip install levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 4.9 MB/s eta 0:00:00


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
def correct_sentence(sentence):
    input_text = "correct: " + sentence
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)

    # Make sure the inputs are on the same device as the model
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}

    outputs = model.generate(inputs['input_ids'])
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected


In [ ]:


sentence_to_correct = "comfotable"
print("sentence:", sentence_to_correct, "\nCorrect sentence:", correct_sentence(sentence_to_correct))


sentence: comfotable 
Correct sentence: comfotable


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import classification_report
import numpy as np
from Levenshtein import distance as levenshtein_distance

def compute_bleu(reference, hypothesis):
    reference = reference.split()
    hypothesis = hypothesis.split()
    return sentence_bleu([reference], hypothesis)

def compute_edit_distance(reference, hypothesis):
    return levenshtein_distance(reference, hypothesis)


bleu_scores = []
edit_distances = []

# Assuming val_df is your validation DataFrame
for index, row in val_df.iterrows():
    if isinstance(row['text'], str) and isinstance(row['corrected'], str):
        predicted = correct_sentence(row['text'])
        bleu_scores.append(compute_bleu(row['corrected'], predicted))
        edit_distances.append(compute_edit_distance(row['corrected'], predicted))

bleu_score =

print(f"Average BLEU Score: {np.mean(bleu_scores)}") #debe estar más cercano a 1
print(f"Average Edit Distance: {np.mean(edit_distances)}")


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


sentence: see u 
Correct sentence: see


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU Score: 2.238748655168301e-232
Average Edit Distance: 3.1559970566593085


In [ ]:

### save metrics as.txt in the same path as the model

# Assuming you have this variable from before
model_output_path = "/content/drive/My Drive/TFM/transformers-models/spellcheckerT5v0"

# Extract model name from the path
model_name = model_output_path.split("/")[-1]

# Create the path for the metrics .txt file
metrics_file_path = f"{model_output_path}/{model_name}_metrics.txt"

# Assuming you have your metrics in a dictionary named `metrics_dict`
metrics_dict = {
    'BLEU': np.mean(bleu_scores),
    'Edit Distance': np.mean(edit_distances),
    # ... add other metrics here
}

# Write the metrics to the .txt file
with open(metrics_file_path, 'w') as file:
    for key, value in metrics_dict.items():
        file.write(f"{key}: {value}\n")


In [ ]:
#save results in a .csv file for all the models and metrics

import pandas as pd
import os

# Assuming you have this variable from before
model_output_path = "/content/drive/My Drive/TFM/transformers-models/spellcheckerT5v0"

# Extract model name from the path
model_name = model_output_path.split("/")[-1]

# Define the path for the metrics .csv file
csv_file_path = "/content/drive/My Drive/TFM/transformers-models/models_metrics.csv"


# Assuming you've calculated these metrics for your model
bleu_score = np.mean(bleu_scores)
edit_distance =np.mean(edit_distances)



# Check if the CSV file already exists
if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
else:
    # Create a new dataframe if the CSV doesn't exist
    df = pd.DataFrame(columns=['Model Name', 'BLEU', 'Levenshtein Distance'])

# Append the new data to the DataFrame
df = df.append({
    'Model Name': model_name,
    'BLEU': bleu_score,
    'Levenshtein Distance': edit_distance
}, ignore_index=True)

# Save the updated DataFrame to the .csv file
df.to_csv(csv_file_path, index=False)


<ipython-input-29-b4fdfa6a4552>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({


In [ ]:
def compute_classification_metrics(reference, hypothesis):
    # Tokenizing words and considering if each word was corrected or not (1 for corrected, 0 for not)
    reference_tokens = reference.split()
    hypothesis_tokens = hypothesis.split()

    y_true = [1 if word != ref else 0 for word, ref in zip(hypothesis_tokens, reference_tokens)]
    y_pred = [1] * len(y_true)  # As our model is always trying to correct

    return y_true, y_pred

all_y_true = []
all_y_pred = []

for index, row in val_df.iterrows():
    if isinstance(row['text'], str) and isinstance(row['corrected'], str):
        predicted = correct_sentence(row['text'])
        y_true, y_pred = compute_classification_metrics(row['corrected'], predicted)
        all_y_true.extend(y_true)
        all_y_pred.extend(y_pred)

print(classification_report(all_y_true, all_y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       168
           1       0.84      1.00      0.92       912

    accuracy                           0.84      1080
   macro avg       0.42      0.50      0.46      1080
weighted avg       0.71      0.84      0.77      1080



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


To use this model in the future

In [ ]:
# TO use this model in the future in the future
# "/content/drive/My Drive/TFM/transformers-models/spellcheckerT5v0"

from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("/content/drive/My Drive/TFM/transformers-models/spellcheckerT5v0")
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Your sentence correction function should work as before
def correct_sentence(sentence):
    input_text = "correct: " + sentence
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(inputs['input_ids'])
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected

sentence = "its your fault"
print(correct_sentence(sentence))


it


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
